In [ ]:
import math
import os

import geopandas as gpd
import pandas as pd
import salem
from dmelon import utils

import xarray as xr

In [ ]:
settings = None

In [ ]:
settings = utils.load_json(settings)

In [ ]:
MONTH = settings["MONTH"]
FYEAR = settings["FYEAR"]
INIT_MONTH = settings["INIT_MONTH"]
DATA_DIR = settings["DATA_DIR"]
MONTH_DIR = os.path.join(DATA_DIR, f"{INIT_MONTH}.{MONTH}")
NC_DIR = os.path.join(DATA_DIR, str(FYEAR), f"{INIT_MONTH}.{MONTH}", "Data")
OUT_DIR = os.path.join(DATA_DIR, str(FYEAR), f"{MONTH}_val_book.xlsx")

In [ ]:
senamhi_sectors = salem.read_shapefile(
    "/data/users/grivera/Shapes/SENAMHI_REGIONS/sectores_climaticos.shp"
)
senamhi_sectors

In [ ]:
pisco = (
    xr.open_dataset(f"{settings['PISCO_DATA'][:-3]}-unstable.nc", decode_times=False)
    .rename({"X": "lon", "Y": "lat", "T": "time"})
    .load()
)
pisco.time.attrs["calendar"] = "360_day"
pisco = xr.decode_cf(pisco).Prec.convert_calendar("standard", align_on="year")
pisco

In [ ]:
pisco_clim = (
    pisco.sel(time=slice("1981-10-01", "2016-10-01")).groupby("time.month").mean("time")
)
pisco_clim

In [ ]:
fcst_data = xr.open_dataset(os.path.join(NC_DIR, "fcst_data.nc")).fcst_data
fcst_data

In [ ]:
def reduce_data_salem(data: xr.DataArray, shape: gpd.GeoDataFrame) -> xr.DataArray:
    reduced = data.salem.roi(shape=shape).mean(dim=["lat", "lon"])
    reduced = reduced.where(reduced > 0, 0)
    return reduced


fcst_sectors = []
clim_sectors = []
pisco_sectors = []
for sector_name in senamhi_sectors["sector"]:
    print(f"{sector_name=}")
    _sector = senamhi_sectors.query("sector == @sector_name")
    _reduced_fcst = reduce_data_salem(fcst_data, _sector)
    _reduced_fcst.name = sector_name

    _reduced_obs = reduce_data_salem(pisco, _sector)
    _reduced_obs.name = sector_name

    _reduced_clim = reduce_data_salem(pisco_clim, _sector)
    _reduced_clim.name = sector_name

    fcst_sectors.append(_reduced_fcst)
    clim_sectors.append(_reduced_clim)
    pisco_sectors.append(_reduced_obs)

fcst_sectors = xr.combine_by_coords(fcst_sectors)
clim_sectors = xr.combine_by_coords(clim_sectors)
obs_sectors = xr.combine_by_coords(pisco_sectors)

In [ ]:
years = range(2016, 2022)

_table_container = {}

for var in fcst_sectors.variables:
    if var == "time":
        continue
    print(f"Entering variable {var=}")
    _df_fcst = fcst_sectors[var].sel(time=slice("2016", None)).to_dataframe()
    _df_clim = (
        clim_sectors[var]
        .roll(month=3, roll_coords=True)
        .to_dataframe()
        .rename(columns={var: f"{var}_clim"})
    )
    _df_obs = obs_sectors[var].sel(time=slice("2016", None)).to_dataframe()

    d = []

    for year in years:
        query = f"(time>='{year}-10-01') & (time<='{year+1}-09-30')"
        _fcts_obs = pd.concat(
            [
                _df_fcst.query(query)
                .rename(columns={var: f"{var}_fcst"})
                .reset_index(drop=True),
                _df_obs.query(query)
                .rename(columns={var: f"{var}_obs"})
                .reset_index(drop=True),
            ],
            axis=1,
        )
        _fcts_obs["month"] = [
            f"{_m:%^b}" for _m in pd.date_range("2022-10-01", periods=12, freq="M")
        ]
        _fcts_obs.set_index("month", inplace=True)
        _fcts_obs.columns = pd.MultiIndex.from_product([[year], _fcts_obs.columns])
        d.append(_fcts_obs)

    _df_clim.index = _fcts_obs.index
    d = pd.concat(d, axis=1)

    _table_container[var] = pd.concat(
        [d, pd.concat({"CLIM": _df_clim}, axis=1)], axis=1
    )

In [ ]:
with pd.ExcelWriter(OUT_DIR) as writer:
    for k, v in _table_container.items():
        v.round(3).to_excel(writer, sheet_name=k)